In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import time
import cv2
import glob
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)

from sklearn.metrics import confusion_matrix
import itertools

import tensorflow as tf
tf.compat.v1.disable_v2_behavior
from tensorflow import keras
from tensorflow.keras.utils import to_categorical # used for converting labels to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dest_dir = os.getcwd() + "/content/drive/MyDrive/Ham10000/ReorganizeD"

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt


print(os.listdir("../content/drive/MyDrive/Ham10000/ReorganizeD"))


['vasc', 'bcc', 'akiec', 'df', 'mel', 'bkl', 'nv']


In [ ]:

import shutil

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
y_train = np.load("/content/drive/MyDrive/SAVE/y_train.npy")
x_train = np.load("/content/drive/MyDrive/SAVE/x_train.npy")

In [ ]:
x_validate = np.load("/content/drive/MyDrive/SAVE/x_validate.npy")
y_validate = np.load("/content/drive/MyDrive/SAVE/y_validate.npy")

In [ ]:
x_test = np.load("/content/drive/MyDrive/SAVE/x_test.npy")
y_test = np.load("/content/drive/MyDrive/SAVE/y_test.npy")

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
\
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#optimizer = SGD(lr=0.001, momentum=15, decay=0.0, nesterov=False)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False, 
        rotation_range=90,  
        zoom_range = 0.1, 
        width_shift_range=0.1, 
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True,  
        shear_range = 10)

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Add
from tensorflow.keras.layers import MaxPool2D, GlobalAvgPool2D, Dense
 
 
def conv_batchnorm_relu(x, filters, kernel_size, strides):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
 
 
def identity_block(tensor, filters):
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
    x = BatchNormalization()(x)
 
    x = Add()([x, tensor])
    x = ReLU()(x)
    return x
 
 
def projection_block(tensor, filters, strides):
    # left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=strides)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
    x = BatchNormalization()(x)
 
    # right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
    shortcut = BatchNormalization()(shortcut)
 
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x
 
 
def resnet_block(x, filters, reps, strides):
    x = projection_block(x, filters=filters, strides=strides)
    for _ in range(reps-1):  # the -1 is because the first block was a Conv one
        x = identity_block(x, filters=filters)
    return x
 
 
input = Input(shape=(256, 256, 3))
 
x = conv_batchnorm_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2
 
x = resnet_block(x, filters=64, reps=3, strides=1)
x = resnet_block(x, filters=128, reps=8, strides=2)  # s=2 ([2]: conv3_1)
x = resnet_block(x, filters=256, reps=36, strides=2)  # s=2 ([2]: conv4_1)
x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)
 
x = GlobalAvgPool2D()(x) 
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x) # [3]: average pool *it is not written any pool size so we use Global
 
output = Dense(7, activation='softmax')(x)  # [3]: 1000-d fc, softmax
 
from tensorflow.keras import Model
 
model = Model(input, output)

In [ ]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_212 (Conv2D)            (None, 128, 128, 64  9472        ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_53 (BatchN  (None, 128, 128, 64  256        ['conv2d_212[0][0]']             
 ormalization)                  )                                                           

In [ ]:
datagen.fit(x_train)
## Fit the model
epochs = 40
batch_size = 64

datagen.fit(x_validate)
history = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size),verbose = 1,
                              epochs = epochs, validation_data = datagen.flow(x_validate,y_validate),callbacks=[learning_rate_reduction])

Epoch 1/40
99/99 [==============================] - 110s 926ms/step - loss: 2.2431 - accuracy: 0.2306 - val_loss: 2.1873 - val_accuracy: 0.1241 - lr: 0.0010
Epoch 2/40
99/99 [==============================] - 89s 899ms/step - loss: 1.5956 - accuracy: 0.3602 - val_loss: 2.3614 - val_accuracy: 0.1241 - lr: 0.0010
Epoch 3/40
99/99 [==============================] - 88s 891ms/step - loss: 1.4472 - accuracy: 0.4255 - val_loss: 2.0723 - val_accuracy: 0.2040 - lr: 0.0010
Epoch 4/40
99/99 [==============================] - 88s 891ms/step - loss: 1.3985 - accuracy: 0.4544 - val_loss: 1.9934 - val_accuracy: 0.2083 - lr: 0.0010
Epoch 5/40
99/99 [==============================] - 89s 895ms/step - loss: 1.3421 - accuracy: 0.4695 - val_loss: 1.8101 - val_accuracy: 0.3680 - lr: 0.0010
Epoch 6/40
99/99 [==============================] - 89s 892ms/step - loss: 1.2903 - accuracy: 0.4874 - val_loss: 1.6249 - val_accuracy: 0.4322 - lr: 0.0010
Epoch 7/40
99/99 [==============================] - 89s 891ms/s

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=0)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

Validation: accuracy = 0.757489  ;  loss_v = 0.696923
Test: accuracy = 0.749429  ;  loss = 0.693171
